In [2]:
import os
# os.environ['MUJOCO_GL'] = 'glfw' #'egl'
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import mujoco
print('MuJoCo version:', mujoco.__version__); 
import mujoco.viewer

MuJoCo version: 3.3.6


In [7]:
import mujoco
import numpy as np

#TODO: adjust inertia position (e.g. heavy motor at femur top is currently not taken into account), 
 #use mass instead of density 
 #nonlinear torque output for knee joint cranklength: real, “0” (see mujoco ref)
 #add armature inertia because motor moves faster than actual joint bc of gearbox 

MODEL_PATH = "leg_hopper.xml"

model = mujoco.MjModel.from_xml_path(MODEL_PATH)
data = mujoco.MjData(model)

# IDs for tendon and actuator
tendon_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_TENDON, "leg_spring")
tendon_actuator_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_ACTUATOR, "leg_spring_actuator")

# Spring parameters
k = 500       # stiffness in N/m
L0 = 0.2109        # rest length in meters, note we set a slightly different rest length because the rest lenhgth is only enforced as soft constraint with 0.001 tolerance 

with mujoco.viewer.launch_passive(model, data) as viewer:
    while viewer.is_running():
        # current tendon length (distance along spring_A → spring_B path)
        L = data.ten_length[tendon_id]

        # spring force only when spring is extended (L > L0)
        if L > L0:
            F = k * (L - L0)
            
        else: 
            F = 0 
            
        print(f"L={L:.4f}[m], F={F:.2f}[N]")

        # apply as actuator control (force along tendon)
        data.ctrl[tendon_actuator_id] = F

        mujoco.mj_step(model, data)
        
        viewer.sync()


L=0.2099[m], F=0.00[N]
L=0.2099[m], F=0.00[N]
L=0.2099[m], F=0.00[N]
L=0.2099[m], F=0.00[N]
L=0.2099[m], F=0.00[N]
L=0.2099[m], F=0.00[N]
L=0.2100[m], F=0.00[N]
L=0.2100[m], F=0.00[N]
L=0.2100[m], F=0.00[N]
L=0.2100[m], F=0.00[N]
L=0.2100[m], F=0.00[N]
L=0.2100[m], F=0.00[N]
L=0.2100[m], F=0.00[N]
L=0.2101[m], F=0.00[N]
L=0.2101[m], F=0.00[N]
L=0.2101[m], F=0.00[N]
L=0.2101[m], F=0.00[N]
L=0.2101[m], F=0.00[N]
L=0.2101[m], F=0.00[N]
L=0.2102[m], F=0.00[N]
L=0.2102[m], F=0.00[N]
L=0.2102[m], F=0.00[N]
L=0.2102[m], F=0.00[N]
L=0.2102[m], F=0.00[N]
L=0.2103[m], F=0.00[N]
L=0.2103[m], F=0.00[N]
L=0.2103[m], F=0.00[N]
L=0.2103[m], F=0.00[N]
L=0.2103[m], F=0.00[N]
L=0.2103[m], F=0.00[N]
L=0.2104[m], F=0.00[N]
L=0.2104[m], F=0.00[N]
L=0.2104[m], F=0.00[N]
L=0.2104[m], F=0.00[N]
L=0.2104[m], F=0.00[N]
L=0.2104[m], F=0.00[N]
L=0.2104[m], F=0.00[N]
L=0.2105[m], F=0.00[N]
L=0.2105[m], F=0.00[N]
L=0.2105[m], F=0.00[N]
L=0.2105[m], F=0.00[N]
L=0.2105[m], F=0.00[N]
L=0.2105[m], F=0.00[N]
L=0.2105[m]

In [34]:
import numpy as np 
import matplotlib.pyplot as plt

#Hopping on the spot
MODEL_PATH = "leg_hopper.xml"

model = mujoco.MjModel.from_xml_path(MODEL_PATH)
data = mujoco.MjData(model)

# IDs for tendon and actuator
tendon_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_TENDON, "leg_spring")
tendon_actuator_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_ACTUATOR, "leg_spring_actuator")

# IDs for hip & knee motors
hip_motor_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_ACTUATOR, "hip_motor")
knee_motor_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_ACTUATOR, "knee_motor")
#ID for tracking 
body_track_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_BODY, "hip_linear_guide")

# (Optional) set an initial crouched configuration so the leg can push
hip_joint_id  = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_JOINT, "hip_joint")
knee_joint_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_JOINT, "knee_joint")

hip_qpos_adr  = model.jnt_qposadr[hip_joint_id]
knee_qpos_adr = model.jnt_qposadr[knee_joint_id]

data.qpos[hip_qpos_adr]  = 0   # rad, small hip flexion
data.qpos[knee_qpos_adr] = 0  # rad, knee flexed

hip_dof_adr = model.jnt_dofadr[hip_joint_id]   # index into qvel for this joint


mujoco.mj_forward(model, data)

#store force 
t_arr, Fs_arr, Ls_arr = [], [], []

# Spring parameters (your values)
k = 700.0          # N/m
L0 = 0.20992       # m spring rest length

# Sinusoid parameters for hopping
f = 3       # Hz (try 1.5–3.0)
# A_hip = 2# hip torque amplitude (within ctrlrange)
A_knee = 2.5      # knee torque amplitude
phi_hip_offset = np.pi - 0.02 # phase offset hip→knee


# PD control parameters for hip pos control 
hip_target = 0.1     # rad (change to whatever angle you want)
Kp_hip = 500.0          # try 1–50 depending on your ctrlrange units
Ki_hip = 200.0
Kd_hip = 50          # try 0.05–5


with mujoco.viewer.launch_passive(model, data) as viewer:
    
    #configure the viewer 
    viewer.cam.type = mujoco.mjtCamera.mjCAMERA_TRACKING
    viewer.cam.trackbodyid = body_track_id

    # optional tuning
    viewer.cam.distance  = 2.5
    viewer.cam.azimuth   = 90
    viewer.cam.elevation = -15
       
        
    # N = 5000  # sim steps to run 
    while viewer.is_running(): #run sim contirnuously 
    # for i in range(N): 
        
        t = data.time

        # --- spring force along the tendon ---
        Ls = data.ten_length[tendon_id]
        # spring force only when spring is extended (L > L0)
        if Ls > L0:
            Fs = k * (Ls - L0)
        else:
            Fs = 0

        # apply spring force along tendon
        data.ctrl[tendon_actuator_id] = Fs
        
        # print(Fs)
        # print(data.time)
        t_arr.append(data.time)
        Ls_arr.append(Ls)
        Fs_arr.append(Fs) 
        
        
        # --- sinusoidal hip & knee torques (open-loop hopping pattern) ---
        # basic pattern: u(t) = A * sin(2π f t + φ)
        u_knee = A_knee * np.sin(2.0 * np.pi * f * t)
        # u_hip  = A_hip  * np.cos(2.0 * np.pi * f * t + phi_hip_offset)
        hip_angle = data.qpos[hip_qpos_adr]
        hip_vel   = data.qvel[hip_dof_adr]
        # print(hip_vel)

        u_hip = Kp_hip * (hip_target - hip_angle) + Kd_hip * (0.0 - hip_vel) 
        print(u_hip)
        

        # write into motor controls (torques)
        data.ctrl[hip_motor_id]  = u_hip
        data.ctrl[knee_motor_id] = u_knee

        # step simulation
        mujoco.mj_step(model, data)
        viewer.sync()
        

# plt.plot(t_arr, Fs_arr)
# plt.title('Spring Force over Time')
# plt.xlabel('Time [s]')
# plt.ylabel('Spring Force [N]')

        
    


50.0
-17.063682784015747
34.68050715487121
-32.19170820204203
29.38036898992793
-37.422523299574365
24.783196994090318
-41.95766810726554
20.840413371380762
-45.84492187462548
17.507502278957325
-40.76877531842276
22.94098481031353
-43.74143059071653
20.464166254415446
-46.16958967087553
18.497417881651216
-43.05244765153128
21.945223005282415
-44.66013382355389
20.730086689278814
-45.83398273316984
19.903530239402944
-46.29902806227163
19.7397958460965
-45.881930157616864
20.413764403735566
-46.05741874720206
20.48342415852834
-45.9557784578919
22.611765641789663
-43.81905768832813
24.763824644520668
-41.65710090889571
26.942325941851557
-39.467318214922514
29.149726685206414
-37.24737530850676
31.388239606646923
-34.995171330271376
33.659854222676
-32.708818603380145
35.966356208745886
-30.386624126048197
38.309345101179765
-28.027072662685356
40.6902504699062
-25.628811296547397
43.11034669320535
-23.19063531844793
45.57076645449142
-20.711475336772736
48.072513070924096
-18.1903855

In [44]:
import mujoco
import numpy as np

MODEL_PATH = "leg_hopper.xml"

model = mujoco.MjModel.from_xml_path(MODEL_PATH)
data = mujoco.MjData(model)

# IDs for tendon and actuator
tendon_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_TENDON, "leg_spring")
tendon_actuator_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_ACTUATOR, "leg_spring_actuator")

# IDs for hip & knee motors
hip_motor_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_ACTUATOR, "hip_motor")
knee_motor_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_ACTUATOR, "knee_motor")
#ID for tracking 
body_track_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_BODY, "hip_linear_guide")

# (Optional) set an initial crouched configuration so the leg can push
hip_joint_id  = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_JOINT, "hip_joint")
knee_joint_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_JOINT, "knee_joint")

hip_qpos_adr  = model.jnt_qposadr[hip_joint_id]
knee_qpos_adr = model.jnt_qposadr[knee_joint_id]

data.qpos[hip_qpos_adr]  = 0   # rad, small hip flexion
data.qpos[knee_qpos_adr] = 0  # rad, knee flexed

mujoco.mj_forward(model, data)

# Spring parameters (your values)
k = 700.0          # N/m
L0 = 0.2109       # m

# Sinusoid parameters for hopping
f = 2        # Hz (try 1.5–3.0)
A_hip = 2# hip torque amplitude (within ctrlrange)
A_knee = 2      # knee torque amplitude
phi_hip_offset = np.pi - 0.02 # phase offset hip→knee

with mujoco.viewer.launch_passive(model, data) as viewer:
    
    #configure the viewer 
    viewer.cam.type = mujoco.mjtCamera.mjCAMERA_TRACKING
    viewer.cam.trackbodyid = body_track_id

    # optional tuning
    viewer.cam.distance  = 0.9
    viewer.cam.azimuth   = 90
    viewer.cam.elevation = -15
    
    while viewer.is_running():

        t = data.time

        # --- spring force along the tendon ---
        L = data.ten_length[tendon_id]
        # spring force only when spring is extended (L > L0)
        if L > L0:
            F = k * (L - L0)
        else:
            F = 0

        # apply spring force along tendon
        data.ctrl[tendon_actuator_id] = F

        # --- sinusoidal hip & knee torques (open-loop hopping pattern) ---
        # basic pattern: u(t) = A * sin(2π f t + φ)
        u_knee = A_knee * np.sin(2.0 * np.pi * f * t)
        u_hip  = A_hip  * np.sin(2.0 * np.pi * f * t + phi_hip_offset)
        

        # write into motor controls (torques)
        data.ctrl[hip_motor_id]  = u_hip
        data.ctrl[knee_motor_id] = u_knee

        # step simulation
        mujoco.mj_step(model, data)
        viewer.sync()


In [41]:
# Standalone RL training cell for YOUR model (leg_hopper.xml)
# - TWO-DOF actuation: hip_motor + knee_motor controlled by the policy
# - Passive tendon spring: F = k*(L-L0) when L>L0 else 0
# - Reward = "best repeatable hop height": encourages multiple hops reaching similar apex height
#
# Requirements:
#   pip install mujoco gymnasium stable-baselines3 numpy imageio imageio-ffmpeg

import os
import numpy as np
import mujoco
import gymnasium as gym
from gymnasium import spaces

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecMonitor, VecVideoRecorder


# ------------------------- USER SETTINGS -------------------------
MODEL_XML_PATH = "leg_hopper.xml"

# Joint/actuator names in your XML
HIP_JOINT_NAME       = "hip_joint"
KNEE_JOINT_NAME      = "knee_joint"
HIP_MOTOR_NAME       = "hip_motor"
KNEE_MOTOR_NAME      = "knee_motor"
TENDON_NAME          = "leg_spring"
TENDON_ACT_NAME      = "leg_spring_actuator"

# Root slider joints (in your XML)
ROOT_X_JOINT_NAME    = "hip_linear_x"
ROOT_Z_JOINT_NAME    = "hip_linear_z"

# Tendon spring parameters (your values)
SPRING_K  = 500.0       # N/m
SPRING_L0 = 0.2109      # m

# Action scaling (must respect each actuator ctrlrange in XML)
# If ctrlrange for hip/knee is [-1, 1], keep these = 1.0.
MAX_HIP_CTRL  = 2.0
MAX_KNEE_CTRL = 2.0

# Env timing
SEED = 0
CTRL_HZ = 50
EPISODE_SECONDS = 3.0    # allow multiple hops

# Repeatable hop reward settings
APEX_WINDOW = 10            # how many recent hops define "repeatable height"
MIN_HOPS_FOR_SCORE = 2     # only start scoring after you have at least this many apex events
CONSISTENCY_W = 4.0        # penalty weight on hop-to-hop variation (higher = more consistent)
HEIGHT_W = 5.0             # reward weight on repeatable height per apex
FLIP_TERMINATE_ANGLE = 2.2 # rad, safety termination if hip joint goes crazy

# PPO
TOTAL_TIMESTEPS = 300_000
SAVE_PATH = "ppo_hip_knee_repeatable_hops.zip"

# Video recording
RECORD_VIDEO = True
VIDEO_DIR = "videos"
VIDEO_EVERY_N_STEPS = 50_000
VIDEO_SECONDS = 20.0
VIDEO_LENGTH = int(VIDEO_SECONDS * CTRL_HZ)

# ---------------------------------------------------------------


def _name2id_or_raise(model, obj_type, name: str) -> int:
    idx = mujoco.mj_name2id(model, obj_type, name)
    if idx < 0:
        raise ValueError(f"Name not found: {name} (type={obj_type})")
    return int(idx)

def _joint_qposadr(model, joint_id: int) -> int:
    return int(model.jnt_qposadr[joint_id])

def _joint_dofadr(model, joint_id: int) -> int:
    return int(model.jnt_dofadr[joint_id])


class HipKneeRepeatHopEnv(gym.Env):
    metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 50}

    def __init__(
        self,
        xml_path: str,
        render_mode: str | None = None,
        episode_seconds: float = 3.0,
        ctrl_hz: int = 50,
        max_hip_ctrl: float = 1.0,
        max_knee_ctrl: float = 1.0,
        seed: int = 0,
    ):
        super().__init__()

        if not os.path.exists(xml_path):
            raise FileNotFoundError(f"MuJoCo XML not found: {xml_path}")

        self.model = mujoco.MjModel.from_xml_path(xml_path)
        self.data = mujoco.MjData(self.model)

        # IDs
        self.hip_jid   = _name2id_or_raise(self.model, mujoco.mjtObj.mjOBJ_JOINT, HIP_JOINT_NAME)
        self.knee_jid  = _name2id_or_raise(self.model, mujoco.mjtObj.mjOBJ_JOINT, KNEE_JOINT_NAME)
        self.rootx_jid = _name2id_or_raise(self.model, mujoco.mjtObj.mjOBJ_JOINT, ROOT_X_JOINT_NAME)
        self.rootz_jid = _name2id_or_raise(self.model, mujoco.mjtObj.mjOBJ_JOINT, ROOT_Z_JOINT_NAME)

        self.hip_aid   = _name2id_or_raise(self.model, mujoco.mjtObj.mjOBJ_ACTUATOR, HIP_MOTOR_NAME)
        self.knee_aid  = _name2id_or_raise(self.model, mujoco.mjtObj.mjOBJ_ACTUATOR, KNEE_MOTOR_NAME)

        self.tendon_id  = _name2id_or_raise(self.model, mujoco.mjtObj.mjOBJ_TENDON, TENDON_NAME)
        self.tendon_aid = _name2id_or_raise(self.model, mujoco.mjtObj.mjOBJ_ACTUATOR, TENDON_ACT_NAME)
        
        self.track_body_id = mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_BODY, "hip_linear_guide")

        # Addresses
        self.hip_qpos_adr   = _joint_qposadr(self.model, self.hip_jid)
        self.hip_dof_adr    = _joint_dofadr(self.model, self.hip_jid)
        self.knee_qpos_adr  = _joint_qposadr(self.model, self.knee_jid)
        self.knee_dof_adr   = _joint_dofadr(self.model, self.knee_jid)
        self.rootz_qpos_adr = _joint_qposadr(self.model, self.rootz_jid)
        self.rootz_dof_adr  = _joint_dofadr(self.model, self.rootz_jid)

        # Timing
        self.episode_seconds = float(episode_seconds)
        self.ctrl_hz = int(ctrl_hz)
        self.ctrl_dt = 1.0 / self.ctrl_hz
        self.sim_dt = float(self.model.opt.timestep)
        self.substeps = max(1, int(round(self.ctrl_dt / self.sim_dt)))

        self.max_hip_ctrl  = float(max_hip_ctrl)
        self.max_knee_ctrl = float(max_knee_ctrl)
        self.render_mode = render_mode
        self.np_random, _ = gym.utils.seeding.np_random(seed)

        self.horizon_steps = int(round(self.episode_seconds * self.ctrl_hz))
        self.t = 0

        # Hop bookkeeping
        self._prev_vz = 0.0
        self._apex_heights = []

        # Spaces
        # obs: [hip_q, hip_qd, knee_q, knee_qd, root_z, root_vz, tendon_L, ncon, phase]
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(9,), dtype=np.float32)

        # action: [hip_u, knee_u] in [-1, 1] each (scaled by MAX_*_CTRL)
        self.action_space = spaces.Box(low=-1.0, high=1.0, shape=(2,), dtype=np.float32)

        # Rendering (rgb_array)
        self._renderer = None

    # ------------------ state helpers ------------------
    def _get_root_z_vz(self) -> tuple[float, float]:
        z  = float(self.data.qpos[self.rootz_qpos_adr])
        vz = float(self.data.qvel[self.rootz_dof_adr])
        return z, vz

    def _get_knee_q_qd(self) -> tuple[float, float]:
        return float(self.data.qpos[self.knee_qpos_adr]), float(self.data.qvel[self.knee_dof_adr])

    def _get_hip_q_qd(self) -> tuple[float, float]:
        return float(self.data.qpos[self.hip_qpos_adr]), float(self.data.qvel[self.hip_dof_adr])

    def _get_tendon_length(self) -> float:
        return float(self.data.ten_length[self.tendon_id])

    def _get_obs(self) -> np.ndarray:
        hip_q, hip_qd = self._get_hip_q_qd()
        knee_q, knee_qd = self._get_knee_q_qd()
        z, vz = self._get_root_z_vz()
        L = self._get_tendon_length()
        ncon = float(self.data.ncon)
        phase = float(self.t) / float(max(1, self.horizon_steps))
        return np.array([hip_q, hip_qd, knee_q, knee_qd, z, vz, L, ncon, phase], dtype=np.float32)

    # ------------------ tendon spring ------------------
    def _apply_tendon_spring_force(self):
        L = float(self.data.ten_length[self.tendon_id])
        F = SPRING_K * (L - SPRING_L0) if L > SPRING_L0 else 0.0
        self.data.ctrl[self.tendon_aid] = float(F)
        return L, float(F)

    # ------------------ hop event detection ------------------
    def _detect_apex_event(self, z: float, vz: float, ncon: int) -> bool:
        apex = (ncon == 0) and (self._prev_vz > 0.0) and (vz <= 0.0)
        self._prev_vz = vz
        if apex:
            self._apex_heights.append(z)
        return apex

    def _repeatable_height_score(self) -> tuple[float, float]:
        if len(self._apex_heights) < MIN_HOPS_FOR_SCORE:
            return 0.0, 0.0
        recent = self._apex_heights[-APEX_WINDOW:]
        repeatable_h = float(np.min(recent))
        std_recent = float(np.std(recent)) if len(recent) >= 2 else 0.0
        return repeatable_h, std_recent

    # ------------------ gym API ------------------
    def reset(self, *, seed=None, options=None):
        super().reset(seed=seed)
        if seed is not None:
            self.np_random, _ = gym.utils.seeding.np_random(seed)

        mujoco.mj_resetData(self.model, self.data)

        # Root start
        self.data.qpos[self.rootz_qpos_adr] = 0.0
        self.data.qvel[self.rootz_dof_adr]  = 0.0

        # Small randomization on both joints (helps robustness)
        self.data.qpos[self.hip_qpos_adr]  += self.np_random.uniform(-0.05, 0.05)
        self.data.qvel[self.hip_dof_adr]   += self.np_random.uniform(-0.2, 0.2)
        self.data.qpos[self.knee_qpos_adr] += self.np_random.uniform(-0.05, 0.05)
        self.data.qvel[self.knee_dof_adr]  += self.np_random.uniform(-0.2, 0.2)

        mujoco.mj_forward(self.model, self.data)

        self.t = 0
        _, vz = self._get_root_z_vz()
        self._prev_vz = vz
        self._apex_heights = []

        return self._get_obs(), {}

    def step(self, action):
        self.t += 1
        self.data.ctrl[:] = 0.0

        # Tendon spring
        L, F = self._apply_tendon_spring_force()

        # Policy actions
        hip_u  = float(np.clip(action[0], -1.0, 1.0)) * self.max_hip_ctrl
        knee_u = float(np.clip(action[1], -1.0, 1.0)) * self.max_knee_ctrl

        self.data.ctrl[self.hip_aid]  = hip_u
        self.data.ctrl[self.knee_aid] = knee_u

        # Physics
        for _ in range(self.substeps):
            mujoco.mj_step(self.model, self.data)

        # State
        z, vz = self._get_root_z_vz()
        ncon = int(self.data.ncon)

        # Reward shaping
        reward = 0.02 * float(z)

        apex = self._detect_apex_event(z, vz, ncon)
        repeatable_h, std_recent = self._repeatable_height_score()

        if apex and len(self._apex_heights) >= MIN_HOPS_FOR_SCORE:
            reward += HEIGHT_W * repeatable_h
            reward -= CONSISTENCY_W * std_recent

        # Control penalty (both joints)
        reward -= 1e-3 * (hip_u * hip_u + knee_u * knee_u)

        # Termination / truncation
        truncated = (self.t >= self.horizon_steps)
        terminated = False

        # Safety: NaNs or extreme fall
        if not np.isfinite(z) or not np.isfinite(vz):
            terminated = True
        if z < -1.5:
            terminated = True

        # Extra safety: hip joint shouldn't blow up
        hip_q, _ = self._get_hip_q_qd()
        if abs(hip_q) > FLIP_TERMINATE_ANGLE:
            terminated = True

        info = {
            "root_z": z,
            "root_vz": vz,
            "tendon_L": L,
            "tendon_F": F,
            "hip_u": hip_u,
            "knee_u": knee_u,
            "ncon": ncon,
            "apex": bool(apex),
            "apex_count": len(self._apex_heights),
            "repeatable_h": repeatable_h,
            "std_recent": std_recent,
            "apex_heights": self._apex_heights.copy(),
        }

        return self._get_obs(), float(reward), terminated, truncated, info

    # ---- Rendering for VecVideoRecorder ----
    def render(self):
        if self.render_mode is None:
            return None
        if self.render_mode == "rgb_array":
            if self._renderer is None:
                self._renderer = mujoco.Renderer(self.model, height=480, width=640)
            self._renderer.update_scene(self.data)
            return self._renderer.render()
        if self.render_mode == "human":
            return None
        raise ValueError(f"Unknown render_mode: {self.render_mode}")

    def close(self):
        if self._renderer is not None:
            try:
                self._renderer.close()
            except Exception:
                pass
            self._renderer = None


# -------------------- Sanity check --------------------
def sanity_check(xml_path: str, steps: int = 300):
    print("=== SANITY CHECK ===")
    env = HipKneeRepeatHopEnv(
        xml_path=xml_path,
        render_mode=None,
        episode_seconds=2.0,
        ctrl_hz=CTRL_HZ,
        max_hip_ctrl=MAX_HIP_CTRL,
        max_knee_ctrl=MAX_KNEE_CTRL,
        seed=SEED
    )
    obs, _ = env.reset(seed=SEED)

    # simple open-loop to verify signals remain sane
    f = 2.0
    A = 0.5
    for i in range(steps):
        t = i / CTRL_HZ
        hip_u  = A * np.sin(2*np.pi*f*t)
        knee_u = A * np.cos(2*np.pi*f*t)
        obs, rew, terminated, truncated, info = env.step(np.array([hip_u, knee_u], dtype=np.float32))

        if i % 50 == 0:
            print(
                f"step={i:4d} | z={info['root_z']:7.4f} | vz={info['root_vz']:7.4f} | "
                f"L={info['tendon_L']:7.4f} | F={info['tendon_F']:7.2f} | "
                f"hip_u={info['hip_u']:6.3f} | knee_u={info['knee_u']:6.3f} | ncon={info['ncon']} | "
                f"apex={int(info['apex'])} | apex_count={info['apex_count']} | repeatable_h={info['repeatable_h']:.4f}"
            )

        if terminated or truncated:
            break

    env.close()
    print("Sanity check done.\n")


def make_env(render_mode=None):
    return HipKneeRepeatHopEnv(
        xml_path=MODEL_XML_PATH,
        render_mode=render_mode,
        episode_seconds=EPISODE_SECONDS,
        ctrl_hz=CTRL_HZ,
        max_hip_ctrl=MAX_HIP_CTRL,
        max_knee_ctrl=MAX_KNEE_CTRL,
        seed=SEED,
    )


# -------------------- 1) Sanity check --------------------
sanity_check(MODEL_XML_PATH, steps=300)

# -------------------- 2) Vector env + video --------------------
base_env = DummyVecEnv([lambda: make_env(render_mode="rgb_array" if RECORD_VIDEO else None)])
vec_env = VecMonitor(base_env)

if RECORD_VIDEO:
    os.makedirs(VIDEO_DIR, exist_ok=True)

    def video_trigger(step_count: int) -> bool:
        return (step_count == 0) or (step_count % VIDEO_EVERY_N_STEPS == 0)

    vec_env = VecVideoRecorder(
        vec_env,
        video_folder=VIDEO_DIR,
        record_video_trigger=video_trigger,
        video_length=VIDEO_LENGTH,
        name_prefix="ppo_hip_knee_repeat_hops"
    )

# -------------------- 3) Train PPO --------------------
model = PPO(
    policy="MlpPolicy",
    env=vec_env,
    n_steps=1024,
    batch_size=256,
    gae_lambda=0.95,
    gamma=0.99,
    n_epochs=10,
    learning_rate=3e-4,
    clip_range=0.2,
    ent_coef=0.0,
    vf_coef=0.5,
    max_grad_norm=0.5,
    verbose=1,
    seed=SEED,
)

model.learn(total_timesteps=TOTAL_TIMESTEPS)
model.save(SAVE_PATH)
print(f"Saved trained policy to: {SAVE_PATH}")

vec_env.close()
print(f"Videos (if enabled) saved in: {VIDEO_DIR}")

# -------------------- 4) Evaluation with hop stats --------------------
eval_env = make_env(render_mode=None)
best_repeatable = -np.inf

for ep in range(10):
    obs, _ = eval_env.reset(seed=SEED + ep)
    done = False
    last_info = None
    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, info = eval_env.step(action)
        done = terminated or truncated
        last_info = info

    apex_heights = last_info["apex_heights"]
    if len(apex_heights) >= MIN_HOPS_FOR_SCORE:
        recent = apex_heights[-APEX_WINDOW:]
        repeatable = float(np.min(recent))
        best_repeatable = max(best_repeatable, repeatable)
        print(f"Episode {ep+1}: apex_count={len(apex_heights)} repeatable_h(min last {APEX_WINDOW})={repeatable:.4f}  recent={np.round(recent,4)}")
    else:
        print(f"Episode {ep+1}: apex_count={len(apex_heights)} (not enough hops to score)")

print(f"Best repeatable height over 10 eval episodes: {best_repeatable:.4f}")

eval_env.close()


=== SANITY CHECK ===
step=   0 | z=-0.0072 | vz=-0.6904 | L= 0.2114 | F=   0.25 | hip_u= 0.000 | knee_u= 1.000 | ncon=0 | apex=0 | apex_count=0 | repeatable_h=0.0000
step=  50 | z=-0.0422 | vz=-0.5338 | L= 0.2148 | F=   1.97 | hip_u=-0.000 | knee_u= 1.000 | ncon=0 | apex=0 | apex_count=3 | repeatable_h=-0.0352
Sanity check done.

Using cpu device
Saving video to g:\My Drive\Study\Hopping_project\simulation\mujoco_leg_hopper\videos\ppo_hip_knee_repeat_hops-step-0-to-step-1000.mp4
MoviePy - Building video g:\My Drive\Study\Hopping_project\simulation\mujoco_leg_hopper\videos\ppo_hip_knee_repeat_hops-step-0-to-step-1000.mp4.
MoviePy - Writing video g:\My Drive\Study\Hopping_project\simulation\mujoco_leg_hopper\videos\ppo_hip_knee_repeat_hops-step-0-to-step-1000.mp4



MoviePy - Done !
MoviePy - video ready g:\My Drive\Study\Hopping_project\simulation\mujoco_leg_hopper\videos\ppo_hip_knee_repeat_hops-step-0-to-step-1000.mp4
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 150      |
|    ep_rew_mean     | -15.8    |
| time/              |          |
|    fps             | 152      |
|    iterations      | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 1024     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 150         |
|    ep_rew_mean          | -15.5       |
| time/                   |             |
|    fps                  | 210         |
|    iterations           | 2           |
|    time_elapsed         | 9           |
|    total_timesteps      | 2048        |
| train/                  |             |
|    approx_kl            | 0.005591372 |
|    clip_fraction        | 0.0416      

MoviePy - Done !
MoviePy - video ready g:\My Drive\Study\Hopping_project\simulation\mujoco_leg_hopper\videos\ppo_hip_knee_repeat_hops-step-50000-to-step-51000.mp4
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 150          |
|    ep_rew_mean          | -1.15        |
| time/                   |              |
|    fps                  | 332          |
|    iterations           | 50           |
|    time_elapsed         | 154          |
|    total_timesteps      | 51200        |
| train/                  |              |
|    approx_kl            | 0.0076335636 |
|    clip_fraction        | 0.0864       |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.57        |
|    explained_variance   | 0.883        |
|    learning_rate        | 0.0003       |
|    loss                 | -0.0129      |
|    n_updates            | 490          |
|    policy_gradient_loss | -0.00965     |
|    std            

MoviePy - Done !
MoviePy - video ready g:\My Drive\Study\Hopping_project\simulation\mujoco_leg_hopper\videos\ppo_hip_knee_repeat_hops-step-100000-to-step-101000.mp4
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 150         |
|    ep_rew_mean          | -0.773      |
| time/                   |             |
|    fps                  | 313         |
|    iterations           | 99          |
|    time_elapsed         | 323         |
|    total_timesteps      | 101376      |
| train/                  |             |
|    approx_kl            | 0.006020223 |
|    clip_fraction        | 0.0385      |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.29       |
|    explained_variance   | 0.638       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0187     |
|    n_updates            | 980         |
|    policy_gradient_loss | -0.00778    |
|    std                  | 0.762    

KeyboardInterrupt: 

In [110]:
import time
import numpy as np
import mujoco.viewer
from stable_baselines3 import PPO

# This 2nd cell is designed to work *directly* with your 1st cell:
# - Uses the same HipKneeRepeatHopEnv class (defined above)
# - Uses the same constants: SAVE_PATH, MODEL_XML_PATH, CTRL_HZ, MAX_HIP_CTRL, MAX_KNEE_CTRL, SEED, FLIP_TERMINATE_ANGLE
# - Runs indefinitely (episode_seconds=1e9) and resets on failure

def run_policy_continuous_viewer_paced(
    *,
    policy,
    xml_path: str,
    ctrl_hz: int,
    max_hip_ctrl: float,
    max_knee_ctrl: float,
    seed: int,
    flip_terminate_angle: float,
    speed: float = 0.25,          # 1.0 = real-time, 0.25 = 4x slower
    reset_on_failure: bool = True,
    print_every_seconds: float = 5.0,
    max_real_fps: float = 60.0,
):
    speed = float(max(1e-6, speed))

    # Use YOUR env class from cell 1
    env = HipKneeRepeatHopEnv(
        xml_path=xml_path,
        render_mode=None,          # viewer handles visualization directly
        episode_seconds=1e9,       # effectively "run forever"
        ctrl_hz=ctrl_hz,
        max_hip_ctrl=max_hip_ctrl,
        max_knee_ctrl=max_knee_ctrl,
        seed=seed,
    )

    try:
        obs, _ = env.reset(seed=seed)

        ctrl_step = 0
        sim_dt_per_ctrl = 1.0 / float(ctrl_hz)

        # Real-time pacing (slow down or speed up the wall-clock)
        target_real_dt_per_ctrl = sim_dt_per_ctrl / speed
        min_real_dt = 1.0 / float(max_real_fps)
        target_real_dt_per_ctrl = max(target_real_dt_per_ctrl, min_real_dt)

        print_every = max(1, int(round(print_every_seconds * ctrl_hz)))

        with mujoco.viewer.launch_passive(env.model, env.data) as viewer:
            while viewer.is_running():
                loop_t0 = time.perf_counter()

                action, _ = policy.predict(obs, deterministic=True)
                obs, reward, terminated, truncated, info = env.step(action)
                ctrl_step += 1

                viewer.sync()

                if (ctrl_step % print_every) == 0:
                    print(
                        f"ctrl={ctrl_step:7d} | z={info.get('root_z', float('nan')):+.4f} | "
                        f"vz={info.get('root_vz', float('nan')):+.4f} | "
                        f"ncon={info.get('ncon', -1)} | apex_cnt={info.get('apex_count', -1)} | "
                        f"repeat_h={info.get('repeatable_h', float('nan')):.4f} | "
                        f"std={info.get('std_recent', float('nan')):.4f} | "
                        f"hip_u={info.get('hip_u', float('nan')):+.3f} | "
                        f"knee_u={info.get('knee_u', float('nan')):+.3f}"
                    )

                # Reset conditions (terminated OR truncated, though truncated shouldn't happen with 1e9)
                if terminated or truncated:
                    # Try to infer a useful reason (matches logic in your env.step)
                    hip_q, _ = env._get_hip_q_qd()
                    root_z = info.get("root_z", np.nan)
                    root_vz = info.get("root_vz", np.nan)

                    reason = "terminated" if terminated else "truncated"
                    if (not np.isfinite(root_z)) or (not np.isfinite(root_vz)):
                        reason = "NaN/Inf state"
                    elif root_z < -1.5:
                        reason = "fell below z < -1.5"
                    elif abs(hip_q) > flip_terminate_angle:
                        reason = f"hip angle blew up |hip_q|={abs(hip_q):.3f} > {flip_terminate_angle:.3f}"

                    print(f"[RESET] {reason} at ctrl={ctrl_step}")

                    if reset_on_failure:
                        obs, _ = env.reset()
                        ctrl_step = 0
                    else:
                        break

                # Wall-clock pacing
                elapsed = time.perf_counter() - loop_t0
                sleep_for = target_real_dt_per_ctrl - elapsed
                if sleep_for > 0:
                    time.sleep(sleep_for)

    finally:
        env.close()


# ---- usage: assumes cell 1 already ran and produced these names ----
# SAVE_PATH, MODEL_XML_PATH, CTRL_HZ, MAX_HIP_CTRL, MAX_KNEE_CTRL, SEED, FLIP_TERMINATE_ANGLE
policy = PPO.load(SAVE_PATH)

run_policy_continuous_viewer_paced(
    policy=policy,
    xml_path=MODEL_XML_PATH,
    ctrl_hz=CTRL_HZ,
    max_hip_ctrl=MAX_HIP_CTRL,
    max_knee_ctrl=MAX_KNEE_CTRL,
    seed=SEED,
    flip_terminate_angle=FLIP_TERMINATE_ANGLE,
    speed=0.2,
)


ctrl=    250 | z=+0.4301 | vz=-0.1769 | ncon=0 | apex_cnt=27 | repeat_h=0.4441 | std=0.0312 | hip_u=-2.000 | knee_u=-2.000
